In [ ]:
from collections import deque
import heapq
import pygame
import sys
import os
from PyQt5 import QtWidgets, QtGui, QtCore
from PyQt5.QtWidgets import QLabel
from PyQt5.QtGui import QPixmap
import timeit

class EightPuzzle:
    def __init__(self, initial, goal):
        self.initial = tuple(map(tuple, initial))
        self.goal = tuple(map(tuple, goal))
        self.rows, self.cols = 3, 3

    def find_blank(self, state):
        for i, row in enumerate(state):
            if 0 in row:
                return i, row.index(0)

    def get_neighbors(self, state):
        row, col = self.find_blank(state)
        moves = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        state_list = [list(row) for row in state]

        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_state = [row[:] for row in state_list]
                new_state[row][col], new_state[new_row][new_col] = new_state[new_row][new_col], new_state[row][col]
                moves.append(tuple(map(tuple, new_state)))
        return moves

    def bfs(self):
        queue = deque([(self.initial, [])])
        visited = {self.initial}

        while queue:
            state, path = queue.popleft()
            if state == self.goal:
                return path + [state]
            for neighbor in self.get_neighbors(state):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, path + [state]))
        return None

    def dfs(self):
        stack = [(self.initial, [])]
        visited = {self.initial}

        while stack:
            state, path = stack.pop()
            if state == self.goal:
                return path + [state]
            for neighbor in self.get_neighbors(state):
                if neighbor not in visited:
                    visited.add(neighbor)
                    stack.append((neighbor, path + [state]))
        return None
    
    def ucs(self):
        open_list = []
        heapq.heappush(open_list, (0, self.initial, []))
        visited = set()
        
        while open_list:
            cost, state, path = heapq.heappop(open_list)
            if state in visited:
                continue
            visited.add(state)
            if state == self.goal:
                return path + [state]
            for neighbor in self.get_neighbors(state):
                if neighbor not in visited:
                    heapq.heappush(open_list, (cost + 1, neighbor, path + [state]))
        return None

    def depth_limited_search(self, state, path, depth, visited):
        if state == self.goal:
            return path + [state]
        if depth == 0:
            return None
        visited.add(state)
        for neighbor in self.get_neighbors(state):
            if neighbor not in visited:
                result = self.depth_limited_search(neighbor, path + [state], depth - 1, visited)
                if result:
                    return result
        return None
    
    def ids(self):
        depth = 0
        while True:
            visited = set()
            result = self.depth_limited_search(self.initial, [], depth, visited)
            if result:
                return result
            depth += 1

    def heuristic(self, state):
        return sum(1 for i in range(3) for j in range(3) if state[i][j] and state[i][j] != self.goal[i][j])

    def greedy_search(self):
        pq = []  
        heapq.heappush(pq, (self.heuristic(self.initial), self.initial, []))  
        visited = set()

        while pq:
            _, state, path = heapq.heappop(pq)

            if state == self.goal:
                return path + [state]

            visited.add(state)

            for neighbor in self.get_neighbors(state):
                if neighbor not in visited:
                    heapq.heappush(pq, (self.heuristic(neighbor), neighbor, path + [state]))

        return None

    def a_star(self): 
        pq = []
        heapq.heappush(pq, (self.heuristic(self.initial), 0, self.initial, []))
        visited = {}

        while pq:
            f, g, state, path = heapq.heappop(pq)

            if state == self.goal:
                return path + [state]

            if state in visited and visited[state] <= g:
                continue

            visited[state] = g

            for neighbor in self.get_neighbors(state):
                new_g = g + 1
                new_f = new_g + self.heuristic(neighbor)
                heapq.heappush(pq, (new_f, new_g, neighbor, path + [state]))

    def ida_star(self):
        threshold = self.heuristic(self.initial)
        
        while True:
            visited = set()
            result, new_threshold = self.ida_star_recursive(self.initial, [], 0, threshold, visited)
            if result:
                return result
            if new_threshold == float('inf'):
                return None
            threshold = new_threshold

    def ida_star_recursive(self, state, path, g, threshold, visited):
        """Hàm hỗ trợ cho IDA*"""
        f = g + self.heuristic(state)
        if f > threshold:
            return None, f
        if state == self.goal:
            return path + [state], threshold
        
        visited.add(state)
        min_threshold = float('inf')

        for neighbor in self.get_neighbors(state):
            if neighbor not in visited:
                result, new_threshold = self.ida_star_recursive(neighbor, path + [state], g + 1, threshold, visited)
                if result:
                    return result, threshold
                min_threshold = min(min_threshold, new_threshold)
        
        visited.remove(state)
        return None, min_threshold

def run_pygame(solution, delay_time, ui_window):
    pygame.init()
    size_x, size_y = 600, 600  
    tile_size = size_x // 3  
    font_size = tile_size // 2  
    font = pygame.font.Font(None, font_size)

    # Lấy vị trí của cửa sổ Qt (UiApp)
    ui_x, ui_y, ui_w, ui_h = ui_window.geometry().getRect()

    # Đặt cửa sổ Pygame ngay bên phải UiApp
    os.environ['SDL_VIDEO_WINDOW_POS'] = f"{ui_x + ui_w},{ui_y}"

    # Khởi tạo cửa sổ Pygame
    screen = pygame.display.set_mode((size_x, size_y))
    pygame.display.set_caption("8-Puzzle Simulation")

    def draw_grid(state):
        screen.fill((255, 255, 255))
        for i, row in enumerate(state):
            for j, num in enumerate(row):
                if num:
                    pygame.draw.rect(screen, (33, 172, 255), (j * tile_size, i * tile_size, tile_size, tile_size))
                    text = font.render(str(num), True, (228, 61, 48))
                    text_rect = text.get_rect(center=(j * tile_size + tile_size // 2, i * tile_size + tile_size // 2))
                    screen.blit(text, text_rect)
        pygame.display.flip()

    for step in solution:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        draw_grid(step)
        pygame.time.delay(delay_time)

    pygame.time.delay(800)
    pygame.quit()

class UiApp(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("8-Puzzle Solver")
        self.setGeometry(300, 300, 600, 600)
        self.setStyleSheet("background-color: white;")  

        # Label tiêu đề
        self.label_title = QtWidgets.QLabel("Bài toán 8-Puzzle Solver", self)
        self.label_title.setGeometry(100, 20, 400, 50)
        self.label_title.setAlignment(QtCore.Qt.AlignCenter)
        self.label_title.setStyleSheet("""
            font-size: 28px;
            font-weight: bold;
            color: #E43D30;  /* Màu đỏ */
            font-family: Arial, sans-serif;
        """)

        # Định dạng chung cho các nút
        button_style = """
            QPushButton {
                font-size: 18px;
                font-weight: bold;
                padding: 10px;
                border-radius: 8px;
                background-color: #1A7DFF; /* Màu xanh dương */
                color: black; /* Chữ đen */
                border: 2px solid #2980b9;
            }
            QPushButton:hover {
                background-color: #2980b9;
            }
        """
        
       
        self.button_bfs = QtWidgets.QPushButton("BFS", self)
        self.button_bfs.setGeometry(50, 100, 160, 60)
        self.button_bfs.setStyleSheet(button_style)
        self.button_bfs.clicked.connect(self.run_bfs)

        self.button_dfs = QtWidgets.QPushButton("DFS", self)
        self.button_dfs.setGeometry(230, 100, 160, 60)
        self.button_dfs.setStyleSheet(button_style)
        self.button_dfs.clicked.connect(self.run_dfs)

        self.button_ucs = QtWidgets.QPushButton("UCS", self)
        self.button_ucs.setGeometry(410, 100, 160, 60)
        self.button_ucs.setStyleSheet(button_style)
        self.button_ucs.clicked.connect(self.run_ucs)

        self.button_ids = QtWidgets.QPushButton("IDS", self)
        self.button_ids.setGeometry(50, 180, 160, 60)
        self.button_ids.setStyleSheet(button_style)
        self.button_ids.clicked.connect(self.run_ids)

        self.button_greedy = QtWidgets.QPushButton("Greedy Search", self)
        self.button_greedy.setGeometry(230, 180, 160, 60)
        self.button_greedy.setStyleSheet(button_style)
        self.button_greedy.clicked.connect(self.run_greedy)

        self.button_astar = QtWidgets.QPushButton("A*", self)
        self.button_astar.setGeometry(410, 180, 160, 60)
        self.button_astar.setStyleSheet(button_style)
        self.button_astar.clicked.connect(self.run_astar)

        self.button_idastar = QtWidgets.QPushButton("IDA*", self)
        self.button_idastar.setGeometry(230, 260, 160, 60)
        self.button_idastar.setStyleSheet(button_style)
        self.button_idastar.clicked.connect(self.run_idastar)

        self.label_time = QtWidgets.QLabel("Thời gian chạy: --- ", self)
        self.label_time.setGeometry(50, 350, 500, 40)
        self.label_time.setAlignment(QtCore.Qt.AlignCenter)
        self.label_time.setStyleSheet("font-size: 24px; color: green; font-weight: bold;")

        self.label_steps = QtWidgets.QLabel("Số bước thực hiện: ---", self)
        self.label_steps.setGeometry(50, 390, 500, 40)
        self.label_steps.setAlignment(QtCore.Qt.AlignCenter)
        self.label_steps.setStyleSheet("font-size: 24px; color: green; font-weight: bold;")

        self.show()
        
    def print_solution(self, solution):
        if solution:
            for step_idx, step in enumerate(solution):
                print(f"\nBước {step_idx}:")
                for row in step:
                    print(row)
            print("\nHoàn thành!")
            return step_idx + 1
        else:
            print("\nKhông tìm thấy lời giải.")
            return 0
            
            
    def run_bfs(self):
        start_time = timeit.default_timer()  
        solution = puzzle.bfs()
        elapsed_time = timeit.default_timer() - start_time  

        if solution:
            run_pygame(solution, 100, self)
            print("\nĐường đi tìm được thuật toán BFS:")
            so_buoc = self.print_solution(solution)
            self.label_time.setText(f"Thời gian chạy: {elapsed_time:.6f} s")
            self.label_steps.setText(f"Số bước thực hiện: {so_buoc}")
  
         

    def run_dfs(self):
        start_time = timeit.default_timer()  
        solution = puzzle.dfs()
        elapsed_time = timeit.default_timer() - start_time  

        if solution:
            run_pygame(solution, 1, self)
            print("\nĐường đi tìm được thuật toán DFS:")
            so_buoc = self.print_solution(solution)
            self.label_time.setText(f"Thời gian chạy: {elapsed_time:.2f} s")
            self.label_steps.setText(f"Số bước thực hiện: {so_buoc}")
        
    def run_ucs(self):
        start_time = timeit.default_timer()  
        solution = puzzle.ucs()
        elapsed_time = timeit.default_timer() - start_time  
        if solution:
            run_pygame(solution, 100, self)
            print("\nĐường đi tìm được thuật toán UCS:")
            so_buoc = self.print_solution(solution)
            self.label_time.setText(f"Thời gian chạy: {elapsed_time:.2f} s")
            self.label_steps.setText(f"Số bước thực hiện: {so_buoc}")
        
    def run_ids(self):
        start_time = timeit.default_timer()  
        solution = puzzle.ids()
        elapsed_time = timeit.default_timer() - start_time  
        if solution:
            run_pygame(solution, 100, self)
            print("\nĐường đi tìm được thuật toán IDS:")
            so_buoc = self.print_solution(solution)
            self.label_time.setText(f"Thời gian chạy: {elapsed_time:.2f} s")
            self.label_steps.setText(f"Số bước thực hiện: {so_buoc}")

    def run_greedy(self):
        start_time = timeit.default_timer()  
        solution = puzzle.greedy_search()
        elapsed_time = timeit.default_timer() - start_time  
        if solution:
            run_pygame(solution, 100, self)
            print("\nĐường đi tìm được thuật toán greedy_search:")
            so_buoc = self.print_solution(solution)
            self.label_time.setText(f"Thời gian chạy: {elapsed_time:.2f} s")
            self.label_steps.setText(f"Số bước thực hiện: {so_buoc}")
            
    def run_astar(self):
        start_time = timeit.default_timer()  
        solution = puzzle.a_star()
        elapsed_time = timeit.default_timer() - start_time  
        if solution:
            run_pygame(solution, 100, self)
            print("\nĐường đi tìm được thuật toán A*:")
            so_buoc = self.print_solution(solution)
            self.label_time.setText(f"Thời gian chạy: {elapsed_time:.2f} s")
            self.label_steps.setText(f"Số bước thực hiện: {so_buoc}")

    def run_idastar(self):
        start_time = timeit.default_timer()  
        solution = puzzle.ida_star()
        elapsed_time = timeit.default_timer() - start_time  
        if solution:
            run_pygame(solution, 100, self)
            print("\nĐường đi tìm được thuật toán IDA*:")
            so_buoc = self.print_solution(solution)
            self.label_time.setText(f"Thời gian chạy: {elapsed_time:.2f} s")
            self.label_steps.setText(f"Số bước thực hiện: {so_buoc}")
    
            

initial_state = [[2, 6, 5], [0, 8, 7], [4, 3, 1]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

puzzle = EightPuzzle(initial_state, goal_state)
app = QtWidgets.QApplication(sys.argv)
window = UiApp()
app.exec_()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html

Đường đi tìm được thuật toán BFS:

Bước 0:
(2, 6, 5)
(0, 8, 7)
(4, 3, 1)

Bước 1:
(2, 6, 5)
(4, 8, 7)
(0, 3, 1)

Bước 2:
(2, 6, 5)
(4, 8, 7)
(3, 0, 1)

Bước 3:
(2, 6, 5)
(4, 8, 7)
(3, 1, 0)

Bước 4:
(2, 6, 5)
(4, 8, 0)
(3, 1, 7)

Bước 5:
(2, 6, 5)
(4, 0, 8)
(3, 1, 7)

Bước 6:
(2, 6, 5)
(4, 1, 8)
(3, 0, 7)

Bước 7:
(2, 6, 5)
(4, 1, 8)
(0, 3, 7)

Bước 8:
(2, 6, 5)
(0, 1, 8)
(4, 3, 7)

Bước 9:
(2, 6, 5)
(1, 0, 8)
(4, 3, 7)

Bước 10:
(2, 6, 5)
(1, 3, 8)
(4, 0, 7)

Bước 11:
(2, 6, 5)
(1, 3, 8)
(4, 7, 0)

Bước 12:
(2, 6, 5)
(1, 3, 0)
(4, 7, 8)

Bước 13:
(2, 6, 0)
(1, 3, 5)
(4, 7, 8)

Bước 14:
(2, 0, 6)
(1, 3, 5)
(4, 7, 8)

Bước 15:
(2, 3, 6)
(1, 0, 5)
(4, 7, 8)

Bước 16:
(2, 3, 6)
(1, 5, 0)
(4, 7, 8)

Bước 17:
(2, 3, 0)
(1, 5, 6)
(4, 7, 8)

Bước 18:
(2, 0, 3)
(1, 5, 6)
(4, 7, 8)

Bước 19:
(0, 2, 3)
(1, 5, 6)
(4, 7, 8)

Bước 20:
(1, 2, 3)
(0, 5, 6)
(4, 7, 8)

Bước 21:
(1, 2, 3)
(4